# Template Search

Given a template image (e.g. `./anchor.png`), can we detect this in the video footage?

![Example template](./anchor.png)

## Part 1: Extract Last Frame From A Video

In [1]:
import cv2
from PIL import Image
import os
import numpy as np 
import os

def extract_last_frame(video_url:str):

     # Read video
     assert os.path.exists(video_url), "Video not found!"
     cap = cv2.VideoCapture(video_url)

     # Get last ms
     fps = cap.get(cv2.CAP_PROP_FPS)
     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
     total_duration_ms = int(total_frames/2)
     last_frame_ms = total_duration_ms - 1  # go to the last millisecond
     print(f"fps {fps} total_frames {total_frames} total_duration_ms {total_duration_ms} last_frame_ms {last_frame_ms}")

     cap.set(cv2.CAP_PROP_POS_MSEC, last_frame_ms)
     ret, last_frame = cap.read()

     while cap.isOpened():
          ret, frame = cap.read()
          if not ret: break
          last_frame = frame

     video_file_seperated, video_file_ext_seperated = os.path.splitext(video_url)
     last_frame_path =  video_file_seperated + "_last" + ".jpg"
     cv2.imwrite(last_frame_path, last_frame)

     cap.release()
     return last_frame_path

In [2]:
def find_files_with_extensions(dir:str, extensions):
    found_files = []
    for root, _, files in os.walk(dir):
        for file in files:
            _, ext = os.path.splitext(file)
            if ext.lower() in [e.lower() for e in extensions]:  # Case-insensitive comparison
                found_files.append(os.path.join(root, file))
    return found_files

In [3]:
movie_files = find_files_with_extensions('.', ['.mov','.mp4'])
frame_files = [extract_last_frame(f) for f in movie_files]

fps 29.994152083556035 total_frames 1378 total_duration_ms 689 last_frame_ms 688
fps 29.96589439021522 total_frames 1274 total_duration_ms 637 last_frame_ms 636
fps 30.0 total_frames 1404 total_duration_ms 702 last_frame_ms 701
fps 60.00212698075082 total_frames 2821 total_duration_ms 1410 last_frame_ms 1409
fps 60.002175568367235 total_frames 2758 total_duration_ms 1379 last_frame_ms 1378
fps 30.00800650622732 total_frames 1373 total_duration_ms 686 last_frame_ms 685
fps 29.983868076716256 total_frames 1394 total_duration_ms 697 last_frame_ms 696
fps 30.0 total_frames 1418 total_duration_ms 709 last_frame_ms 708
fps 60.00132016105965 total_frames 2727 total_duration_ms 1363 last_frame_ms 1362
fps 60.00218269125832 total_frames 2749 total_duration_ms 1374 last_frame_ms 1373
fps 30.00832465367682 total_frames 1365 total_duration_ms 682 last_frame_ms 681
fps 29.97476502212632 total_frames 1366 total_duration_ms 683 last_frame_ms 682
fps 30.0 total_frames 1419 total_duration_ms 709 last_f

## Part 2: Template Matching

In [ ]:
def find_template_match(
    frame_filename, 
    template_filename, 
    min_size=10, 
    max_size=50, 
        delta_size=5, 
        thresh=0.9,
        draw_bbox=False,
        draw_centers=True,
        bbox_color=[0,255,255],
        bbox_thickness=1,
        verbose=False):
    
    # Load frame using opencv
    frame = cv2.imread(frame_filename)
    
    # Load template using opencv
    template_all = cv2.imread(template_filename, cv2.IMREAD_UNCHANGED)
    # Prep boxes list
    boxes = []
    # Iterate through possible sizes of the template, upwards to half of the size
    for p in np.arange(min_size, max_size, delta_size):
        # Resize the frame
        template_resize = cv2.resize(template_all, (p,p))
        # Get particular attributes of the image itself. 
        # We assume transparency, so we have to separate alpha from bgr
        template = template_resize[:,:,0:3]
        alpha = template_resize[:,:,3]
        alpha = cv2.merge([alpha,alpha,alpha])
        # get the width and height of the template
        h,w = template.shape[:2]
        # Prepare possible locations where the template matches
        loc = []
        # Find those matches.
        res = cv2.matchTemplate(
            frame,
            template,
            cv2.TM_CCORR_NORMED,
            mask=alpha
        )
        # threshold by a 
        loc = np.where(res >= thresh)
        if len(loc) > 0:
            for pt in zip(*loc[::-1]):
                boxes.append((pt[0],pt[1],pt[0]+w,pt[1]+h, pt[0]+(w/2), pt[1]+(h/2)))

    
    centers = []
    for (x1, y1, x2, y2, cx, cy) in boxes:
        #result = cv2.rectangle(result, (x1, y1), (x2, y2), bbox_color, bbox_thickness)
        centers.append([cx,cy])

    mean_center = np.mean(centers, axis=0)
    median_center = np.median(centers, axis=0)
    
    if verbose:
        print(f"ESTIMATED MEAN POSITION: {mean_center}")
        print(f"ESTIMATED MEDIAN POSITION: {median_center}")
    
    if draw_bbox or draw_centers:
        result = frame.copy()
        if draw_bbox:
             for (x1, y1, x2, y2, cx, cy) in boxes:
                 result = cv2.rectangle(result, (x1, y1), (x2, y2), bbox_color, bbox_thickness)
        if draw_centers:
            result = cv2.drawMarker(result, (int(mean_center[0]), int(mean_center[1])), (0,255,255),cv2.MARKER_CROSS,20,2)
            result = cv2.drawMarker(result, (int(median_center[0]), int(median_center[1])), (255,255,0),cv2.MARKER_TILTED_CROSS,20,2)

        # Save resulting image
        frame_file_seperated, _ = os.path.splitext(frame_filename)
        result_path =  frame_file_seperated + "_matched" + ".jpg"
        cv2.imwrite(result_path, result)

        return mean_center, median_center, result
    
    return mean_center, median_center

In [11]:
_TEMPLATE = './anchor.png'
last_centers = [find_template_match(f, _TEMPLATE, verbose=True) for f in frame_files]

ESTIMATED MEAN POSITION: [1321.48392819  736.82544071]
ESTIMATED MEDIAN POSITION: [1321.5  737. ]
ESTIMATED MEAN POSITION: [640.80245894 638.72119766]
ESTIMATED MEDIAN POSITION: [641. 639.]
ESTIMATED MEAN POSITION: [1515.58717088 1027.43089696]
ESTIMATED MEDIAN POSITION: [1514. 1019.]


KeyboardInterrupt: 